In [6]:
'''from google.colab import drive
drive.mount('/content/drive')
!ls "/content/drive/My Drive/Colab Notebooks/monthly_20d"
'''

'from google.colab import drive\ndrive.mount(\'/content/drive\')\n!ls "/content/drive/My Drive/Colab Notebooks/monthly_20d"\n'

In [7]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 14507771219749973033]

In [8]:
import tensorflow as tf
tf.test.gpu_device_name()

''

In [9]:
import pandas as pd

In [10]:
import pandas as pd
import numpy as np
import os.path as op
import os

from sklearn.metrics import confusion_matrix, classification_report, cohen_kappa_score

IMAGE_WIDTH = {5: 15, 20: 60, 60: 180}
IMAGE_HEIGHT = {5: 32, 20: 64, 60: 96}  

In [11]:
images_dev = np.memmap(op.join("monthly_20d", f"20d_month_has_vb_[20]_ma_{1993}_images.dat"), 
                         dtype=np.uint8, mode='r').\
                        reshape((-1, IMAGE_HEIGHT[20], IMAGE_WIDTH[20]))
for year in range(1994, 2000):
    images_dev = np.concatenate((images_dev, 
                                   np.memmap(op.join("monthly_20d", 
                                                     f"20d_month_has_vb_[20]_ma_{year}_images.dat"), 
                                                         dtype=np.uint8, mode='r').\
                                   reshape((-1, IMAGE_HEIGHT[20], IMAGE_WIDTH[20])) 
                                                         ))
print(images_dev.shape)

(694871, 64, 60)


In [12]:
images_valid = np.memmap(op.join("monthly_20d", f"20d_month_has_vb_[20]_ma_{2001}_images.dat"), 
                         dtype=np.uint8, mode='r').\
                        reshape((-1, IMAGE_HEIGHT[20], IMAGE_WIDTH[20]))
for year in range(2002, 2020):
    images_valid = np.concatenate((images_valid, 
                                   np.memmap(op.join("monthly_20d", 
                                                     f"20d_month_has_vb_[20]_ma_{year}_images.dat"), 
                                                         dtype=np.uint8, mode='r').\
                                   reshape((-1, IMAGE_HEIGHT[20], IMAGE_WIDTH[20])) 
                                                         ))
print(images_valid.shape)


(1403975, 64, 60)


In [13]:
label_df_dev = pd.read_feather(op.join("monthly_20d", f"20d_month_has_vb_[20]_ma_{1993}_labels_w_delay.feather"))
for year in range(1994, 2000):
    label_df_dev = pd.concat([label_df_dev,
                            pd.read_feather(op.join("monthly_20d", f"20d_month_has_vb_[20]_ma_{year}_labels_w_delay.feather"))
                            ], ignore_index = True)


label_df_valid = pd.read_feather(op.join("monthly_20d", f"20d_month_has_vb_[20]_ma_{2001}_labels_w_delay.feather"))
for year in range(2002, 2020):
    label_df_valid = pd.concat([label_df_valid,
                            pd.read_feather(op.join("monthly_20d", f"20d_month_has_vb_[20]_ma_{year}_labels_w_delay.feather"))
                            ], ignore_index = True)

print (label_df_dev.shape, label_df_valid.shape)

(694871, 35) (1403975, 35)


In [14]:
import torch
from torch import nn
from torch import optim
import torch.utils.data as data
import torch.nn.functional as F

In [25]:
class ConvNet(nn.Module):
    def __init__(self, in_features, class_num):
        super(ConvNet, self).__init__()
        # set size
        self.in_features = in_features
        self.class_num = class_num 
        #remove stride because of input size too small
        self.conv_1 = nn.Sequential(nn.Conv2d(in_channels=1,
                                              out_channels=64,
                                              kernel_size=(5,3),
                                              stride = (2,1),
                                              padding=0,
                                              dilation= (2,1),
                                              bias=False),
                                    nn.LeakyReLU(negative_slope=0.01, inplace=True), #default is 0.01
                                    nn.MaxPool2d(kernel_size=(2,1), stride = (1,1)))
        self.conv_2 = nn.Sequential(nn.Conv2d(in_channels=64,
                                              out_channels=128,
                                              kernel_size=(5,3),
                                              stride = (2,1),
                                              padding=0,
#                                               dilation= (2,1),
                                              bias=False),
                                    nn.LeakyReLU(negative_slope=0.01, inplace=True), #default is 0.01
                                    nn.MaxPool2d(kernel_size=(2,1), stride = (1,1)))
        self.conv_3 = nn.Sequential(nn.Conv2d(in_channels=128,
                                              out_channels=256,
                                              kernel_size=(5,3),
                                              stride = (2,1),
                                              padding=0,
#                                               dilation= (2,1),
                                              bias=False),
                                    nn.LeakyReLU(negative_slope=0.01, inplace=True), #default is 0.01
                                    nn.MaxPool2d(kernel_size=(2,1), stride = (1,1)))
        self.linear = nn.Linear(in_features=self.in_features, out_features=self.class_num)
        self.softm = nn.Softmax(dim=-1) 

    def forward(self, x):
#         print(x.shape, 1)
        conv_feat = self.conv_1(x)
#         print(conv_feat.shape, 2)            
        conv_feat = self.conv_2(conv_feat)
#         print(conv_feat.shape, 3) 
        conv_feat = self.conv_3(conv_feat)
#         print(conv_feat.shape, 4)  
        conv_feat = conv_feat.view(conv_feat.shape[0], -1)
#         print(conv_feat.shape, 5)  
#         print (conv_feat)
        conv_feat  = self.linear(conv_feat)
#         print (conv_feat)

#         y = self.softm(conv_feat)
        #y = torch.unsqueeze(y, 0)
        #print(y) 
        return conv_feat

In [16]:
class SequentialDataLoader(data.Dataset):
    def __init__(self, data_tup,
            partition='Train'):
        super(SequentialDataLoader, self).__init__()
        self.data_tup = data_tup
        self.X = self.data_tup[0]
        self.Y = self.data_tup[1]
        self.partition = partition         
        
        
        self.data_len = self.X.shape[0]
       

    def get_data_element(self,index):
        data_element_x = [self.X[index]]
        data_element_y = self.Y['Retx_20d_label'][index] 
        data_element_x = torch.from_numpy(np.asarray(data_element_x)).float()
        data_element_y = torch.tensor(np.asarray(data_element_y)).long().view(-1)
        #print (data_element_y)
        data_element_x = data_element_x
        data_element_y = data_element_y

        return data_element_x, data_element_y

    def __len__(self):
        return self.data_len

    def __getitem__(self, index):
        return self.get_data_element(index)

In [35]:
class ModelTrainer(object):
    def __init__(self,
                dataloader):
        
        # set encoder and gnn
        self.infeatures = 41472
        self.model = ConvNet(in_features=self.infeatures, class_num=3)
        self.model = torch.nn.DataParallel(self.model, device_ids=[0]).cuda()
        cuda_device = 'cuda:'+str(self.model.device_ids[0])
        self.model.to(cuda_device)

        # get data 
        self.data_loader = dataloader 
        
        # set optimizer
        base_lr = 0.0001
        weight_decay = 1e-4
#         self.optimizer = optim.SGD(self.model.parameters(), lr=base_lr, momentum=0.9)
#         self.optimizer = optim.RMSprop(self.model.parameters(), lr=base_lr, alpha = 0.9)
        self.optimizer = optim.Adam(self.model.parameters(), lr=base_lr,betas = (0.9,0.99))



        # set loss
        self.weights     = [1.0,1.0,0.1]
        self.weights     = torch.from_numpy(np.asarray(self.weights)).float().cuda()  
        self.criterion   = torch.nn.CrossEntropyLoss(weight=self.weights).cuda()
        #self.criterion   = nn.CrossEntropyLoss().cuda()

        self.val_acc = 0
        self.test_acc = 0

    def train(self):
        val_acc = self.val_acc

        for epoch in range(0, 10):
            self.model.train()
            for i, (data, target) in enumerate(self.data_loader['train']):
#                 print (i)
                # init grad
                self.optimizer.zero_grad()
                data   = data.cuda(non_blocking=True)
                target = target.cuda(non_blocking=True).long().contiguous().view(-1)
                score  = self.model(data).float()
                
                #print (score)
                #print (target)
                loss   = self.criterion(score, target) 
                
                loss.backward()
                self.optimizer.step()

            # evaluation
            
            train_acc = self.eval(epoch,partition='train')            
            val_acc = self.eval(epoch,partition='test')
            
#             print('Loss: '+str(loss))
            
            is_best = 0

            if val_acc >= self.val_acc:
                self.val_acc = val_acc
                is_best = 1
            
            #tt.log_step(global_step=self.global_step)

    def eval(self, epoch, partition='test', log_flag=True):
        best_acc = 0

        self.model.eval()
        valy_pred = []
        valy_test = [] 
        valy      = []                
        for i, (data, target) in enumerate(self.data_loader[partition]):
            data   = data.cuda(non_blocking=True)
            target = target.cuda(non_blocking=True).long().contiguous().view(-1)
            
            score  = self.model(data)
            #score  = nn.functional.softmax(score, dim=1) 
            
            loss   = self.criterion(score, target) 

            valy_pred.append(score.detach())
            valy_test.append(target.detach())

            target = self.one_hot_encode(3,target) 
            valy.append(target.detach())
        
        valy_pred = torch.cat(valy_pred,dim=0).detach().cpu().numpy()
        valy_test = torch.cat(valy_test,dim=0).detach().cpu().numpy() 
        valy      = torch.cat(valy,dim=0).detach().cpu().numpy()

        
        # How well have we done on test data
        print("===============================================\r\n")
        print("The %d-th epoch" + str(epoch) + "\r\n")
        print(partition + "\r\n")
        Y=np.argmax(valy,axis=1)
        Yhat=np.argmax(valy_pred,axis=1)
        
#         c=confusion_matrix(Y,Yhat)
#         c=np.concatenate((c,np.sum(c,axis=1).reshape(-1,1)),axis=1)
#         c=np.concatenate((c,np.sum(c,axis=0).reshape(1,-1)),axis=0)
#         print(str(c) + "\r\n")
        print(str(classification_report(Yhat,Y)) + "\r\n")
        print('Loss: '+str(loss))
#         print("cohen kappa score:",cohen_kappa_score(Yhat,Y), "\r\n")
#         print("cohen kappa score:" + str(cohen_kappa_score(Yhat,Y)) + "\r\n")
       
                 
        return cohen_kappa_score(Yhat,Y)


    def one_hot_encode(self, num_classes, class_idx):
        return torch.eye(num_classes)[class_idx].to(0)

In [18]:
os.environ['CUDA_VISIBLE_DEVICES'] = "1"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(torch.cuda.is_available())

True


In [19]:
# split the dev data into train and test
train_index = np.random.choice(images_dev.shape[0],int(images_dev.shape[0]*.7),replace = False)

images_train=images_dev[train_index]
label_df_train = label_df_dev.loc[train_index].reset_index()

all_index = [i for i in range(images_dev.shape[0])]
test_index = list(set(all_index)-set(train_index))

images_test=images_dev[test_index]
label_df_test = label_df_dev.loc[test_index].reset_index()

train_dataset= SequentialDataLoader(data_tup=(images_train, label_df_train),   
                                  partition='Train')
train_loader = torch.utils.data.DataLoader(train_dataset,
                                  batch_size=100,
                                  shuffle=True,
                                  num_workers=2)

test_dataset= SequentialDataLoader(data_tup=(images_test, label_df_test),   
                                  partition='Test')
test_loader = torch.utils.data.DataLoader(test_dataset,
                                  batch_size=100,
                                  shuffle=True,
                                  num_workers=2)

valid_dataset= SequentialDataLoader(data_tup=(images_valid, label_df_valid),   
                                  partition='Valid')
valid_loader = torch.utils.data.DataLoader(valid_dataset,
                                  batch_size=1000,
                                  shuffle=True,
                                  num_workers=2)

dev_data_loader = {'train': train_loader,
               'test': test_loader}

valid_data_loader = {'train': train_loader,
              'test': valid_loader}

In [36]:
trainer = ModelTrainer(dataloader=dev_data_loader)

#%%
trainer.train()


The %d-th epoch0

train



/home/mafs6010g20/anaconda3/envs/ai_finance/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mafs6010g20/anaconda3/envs/ai_finance/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mafs6010g20/anaconda3/envs/ai_finance/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(r

              precision    recall  f1-score   support

           0       0.15      0.55      0.23     65937
           1       0.87      0.49      0.63    420472
           2       0.00      0.00      0.00         0

    accuracy                           0.50    486409
   macro avg       0.34      0.34      0.29    486409
weighted avg       0.78      0.50      0.57    486409


Loss: tensor(0.6718, device='cuda:0', grad_fn=<NllLossBackward0>)

The %d-th epoch0

test



/home/mafs6010g20/anaconda3/envs/ai_finance/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mafs6010g20/anaconda3/envs/ai_finance/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.14      0.54      0.23     28611
           1       0.87      0.49      0.63    179851
           2       0.00      0.00      0.00         0

    accuracy                           0.49    208462
   macro avg       0.34      0.34      0.28    208462
weighted avg       0.77      0.49      0.57    208462


Loss: tensor(0.6935, device='cuda:0', grad_fn=<NllLossBackward0>)


...

/home/mafs6010g20/anaconda3/envs/ai_finance/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mafs6010g20/anaconda3/envs/ai_finance/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mafs6010g20/anaconda3/envs/ai_finance/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(r

              precision    recall  f1-score   support

           0       0.56      0.56      0.56    248054
           1       0.54      0.53      0.54    238355
           2       0.00      0.00      0.00         0

    accuracy                           0.55    486409
   macro avg       0.37      0.36      0.37    486409
weighted avg       0.55      0.55      0.55    486409


Loss: tensor(0.6794, device='cuda:0', grad_fn=<NllLossBackward0>)

The %d-th epoch9

test



/home/mafs6010g20/anaconda3/envs/ai_finance/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mafs6010g20/anaconda3/envs/ai_finance/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.53      0.53      0.53    106002
           1       0.52      0.51      0.51    102460
           2       0.00      0.00      0.00         0

    accuracy                           0.52    208462
   macro avg       0.35      0.35      0.35    208462
weighted avg       0.52      0.52      0.52    208462


Loss: tensor(0.6881, device='cuda:0', grad_fn=<NllLossBackward0>)


/home/mafs6010g20/anaconda3/envs/ai_finance/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [37]:
# !cat /proc/meminfo

In [38]:
# import gc
# gc.collect()

In [39]:
# out of time validation
validator = ModelTrainer(dataloader=valid_data_loader)

#%%
validator.train()


The %d-th epoch0

train



/home/mafs6010g20/anaconda3/envs/ai_finance/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mafs6010g20/anaconda3/envs/ai_finance/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mafs6010g20/anaconda3/envs/ai_finance/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(r

              precision    recall  f1-score   support

           0       0.77      0.52      0.62    366559
           1       0.26      0.51      0.35    119850
           2       0.00      0.00      0.00         0

    accuracy                           0.52    486409
   macro avg       0.34      0.34      0.32    486409
weighted avg       0.64      0.52      0.55    486409


Loss: tensor(0.7955, device='cuda:0', grad_fn=<NllLossBackward0>)

The %d-th epoch0

test



/home/mafs6010g20/anaconda3/envs/ai_finance/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mafs6010g20/anaconda3/envs/ai_finance/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mafs6010g20/anaconda3/envs/ai_finance/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(r

              precision    recall  f1-score   support

           0       0.76      0.49      0.60   1052306
           1       0.26      0.52      0.34    351669
           2       0.00      0.00      0.00         0

    accuracy                           0.50   1403975
   macro avg       0.34      0.34      0.31   1403975
weighted avg       0.63      0.50      0.53   1403975


Loss: tensor(0.6992, device='cuda:0', grad_fn=<NllLossBackward0>)

The %d-th epoch1

train



...

              precision    recall  f1-score   support

           0       0.61      0.50      0.55    835713
           1       0.42      0.52      0.46    568262
           2       0.00      0.00      0.00         0

    accuracy                           0.51   1403975
   macro avg       0.34      0.34      0.34   1403975
weighted avg       0.53      0.51      0.51   1403975


Loss: tensor(0.7034, device='cuda:0', grad_fn=<NllLossBackward0>)

The %d-th epoch9

train



/home/mafs6010g20/anaconda3/envs/ai_finance/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mafs6010g20/anaconda3/envs/ai_finance/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mafs6010g20/anaconda3/envs/ai_finance/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(r

              precision    recall  f1-score   support

           0       0.47      0.57      0.51    202941
           1       0.64      0.53      0.58    283468
           2       0.00      0.00      0.00         0

    accuracy                           0.55    486409
   macro avg       0.37      0.37      0.36    486409
weighted avg       0.57      0.55      0.55    486409


Loss: tensor(0.7129, device='cuda:0', grad_fn=<NllLossBackward0>)

The %d-th epoch9

test



/home/mafs6010g20/anaconda3/envs/ai_finance/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mafs6010g20/anaconda3/envs/ai_finance/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mafs6010g20/anaconda3/envs/ai_finance/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(r

              precision    recall  f1-score   support

           0       0.38      0.51      0.43    510217
           1       0.65      0.52      0.57    893758
           2       0.00      0.00      0.00         0

    accuracy                           0.51   1403975
   macro avg       0.34      0.34      0.34   1403975
weighted avg       0.55      0.51      0.52   1403975


Loss: tensor(0.7013, device='cuda:0', grad_fn=<NllLossBackward0>)
